## IBGE Data Analysis Project

IBGE is the main source for all kinds os statistical data in Brazil. This repository is an exploration of interesting socioeconomic data as well as concepts like Data visualization, REST API abstraction and Test driven development applied to Data Science.

In [106]:
import urllib3
#import json
import matplotlib.pyplot as plot
import numpy
import pandas
from ibge_lib import *


In [107]:
http = urllib3.PoolManager()
http_fn = lambda url: http.request('GET', url)

aggrs_query_url = aggregates_url()
researches = query_ibge(aggrs_query_url, http_fn)

 ### Picking an aggregate

 IBGE's API needs the exact data aggregate code in order to supply any useful data. Therefore, we need to ask for the list of aggregates and search for the code of the aggregate we want. Luckly, these are grouped by research. In this case, we want the National Household Sampling Research (PNAD).

In [108]:
format_fn = lambda x: f"{x['id']}: {x['nome']}"
#list(map(format_fn, researches))

In [109]:
pnad_id = 'B5'
filter_pnad_fn = lambda x: x['id'] == pnad_id
pnad_metadata = next(filter(filter_pnad_fn, researches)) # 'next' gets the first element, in this case
#pnad_data

In [110]:
filtered_aggr = filter_aggregates(pnad_metadata, 'rendimento domiciliar per capita')
format_fn = lambda aggr: f"{aggr['id']}: {aggr['nome']}"
#list(map(format_fn, filtered_aggr))

In [111]:
income_upper_bounds_aggr = 7438
upper_bounds_metadata = query_ibge( metadata_url(income_upper_bounds_aggr), http_fn )
#upper_bounds_metadata

In [112]:
upper_boounds_var = '10769'
upper_bounds_classif = '1019'
uppper_bounds_classes = [ str(x['id']) for x in upper_bounds_metadata['classificacoes'][0]['categorias'] ]
uppper_bounds_classes_fmt = f'[{",".join(uppper_bounds_classes)}]'
#uppper_bounds_classes_fmt

In [113]:
var_query_url = variables_url( income_upper_bounds_aggr, [upper_boounds_var], [f'{upper_bounds_classif}{uppper_bounds_classes_fmt}'])
upper_bounds_response = query_ibge(var_query_url, http_fn )
#upper_bounds_response

In [114]:
income_upper_bounds_results = upper_bounds_response[0]['resultados'][1:]
income_upper_bounds_serii = map( (lambda x: x['series'][0]['serie']) , income_upper_bounds_results)
# income_upper_bounds_results
income_upper_bounds_serii = list(income_upper_bounds_serii)
#income_upper_bounds_series

In [115]:
income_percentiles = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99]
bound_columns = list(map( (lambda x: f'P{x}'), income_percentiles ))
years_queried = list(income_upper_bounds_serii[0].keys())

In [122]:
series_to_row_fn = lambda series: list(map(int, series.values()))
upper_bounds_data = list(map( series_to_row_fn , income_upper_bounds_serii ))
upper_bounds_data = numpy.matrix(upper_bounds_data).transpose()
pandas.DataFrame( upper_bounds_data , index=years_queried, columns=bound_columns)

,P5,P10,P20,P30,P40,P50,P60,P70,P80,P90,P95,P99
2015,123,198,306,411,529,672,814,1029,1380,2177,3353,7758
2016,114,197,307,433,552,696,875,1082,1479,2382,3714,8658
2017,102,199,316,451,578,730,933,1130,1511,2453,3821,9049
2018,103,201,334,473,610,773,967,1211,1641,2625,4067,9893
2019,107,211,348,498,650,826,1002,1282,1733,2723,4209,10365
2020,146,252,396,527,673,836,1035,1263,1682,2646,4082,9832


In [117]:
cumm_income_dist_aggr = 7530
avg_income_per_class_aggr = 7532

In [118]:
cumm_dist_metadata = query_ibge( metadata_url(cumm_income_dist_aggr), http_fn )
income_per_class_metadata = query_ibge( metadata_url(avg_income_per_class_aggr), http_fn )

In [119]:
X = income_percentiles
widths = numpy.array([-5,  -5, -10, -10, -10, -10, -10, -10, -10, - 10,  -5,  -4,   -1])
Y = numpy.array(income_dist)/(-widths)

fig, ax = plot.subplots()
ax.bar(X, Y, align='edge', width=widths)

NameError: name 'income_dist' is not defined